In [26]:
// Import necessary dependencies
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.13.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-serialization-json:1.5.1")



In [36]:
import com.fasterxml.jackson.annotation.JsonFormat
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.ConstraintCollectors
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import java.io.File
import java.time.Duration
import java.time.LocalTime


In [28]:
import com.fasterxml.jackson.annotation.JsonProperty

data class Client(
    @JsonProperty("Client_ID")
    val clientId: String,

    @JsonProperty("Need")
    val need: String, // "Waste", "Enrobé", "Grue"

    @JsonProperty("Volume_t")
    val volumeT: Double,

    @JsonProperty("Volume_kg")
    val volumeKg: Double,

    @JsonProperty("Access_Difficulty")
    val accessDifficulty: String, // "Yes" or "No"

    @JsonProperty("Time_Window_Label")
    val timeWindowLabel: String, // "Morning", "Afternoon", "All Day"

    @JsonProperty("Time_Window_Start")
    val timeWindowStart: String, // "08:00", "13:00", etc.

    @JsonProperty("Time_Window_End")
    val timeWindowEnd: String,

    @JsonProperty("Latitude")
    val latitude: Double,

    @JsonProperty("Longitude")
    val longitude: Double
) {
    val location: Location = Location(latitude, longitude)

    // Convert "Yes"/"No" to Boolean
    val hasAccessDifficulty: Boolean
        get() = accessDifficulty.equals("Yes", ignoreCase = true)

    override fun toString(): String = clientId
}


In [29]:
val clientsJson = """


[
    {
        "Client_ID":"C001",
        "Need":"Grue",
        "Volume_t":13.73,
        "Volume_kg":13726.61,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8727114265,
        "Longitude":2.4152913367
    },
    {
        "Client_ID":"C002",
        "Need":"Enrobée",
        "Volume_t":16.51,
        "Volume_kg":16511.92,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8337749631,
        "Longitude":2.4624881962
    },
    {
        "Client_ID":"C003",
        "Need":"Grue",
        "Volume_t":14.97,
        "Volume_kg":14966.65,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8995203229,
        "Longitude":2.3830675345
    },
    {
        "Client_ID":"C004",
        "Need":"Grue",
        "Volume_t":16.41,
        "Volume_kg":16406.54,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8552772065,
        "Longitude":2.3719840732
    },
    {
        "Client_ID":"C005",
        "Need":"Waste",
        "Volume_t":10.42,
        "Volume_kg":10417.48,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8267392211,
        "Longitude":2.3840461925
    },
    {
        "Client_ID":"C006",
        "Need":"Enrobée",
        "Volume_t":17.52,
        "Volume_kg":17524.55,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8798084826,
        "Longitude":2.2785357718
    },
    {
        "Client_ID":"C007",
        "Need":"Enrobée",
        "Volume_t":17.49,
        "Volume_kg":17491.73,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8424366831,
        "Longitude":2.3312467777
    },
    {
        "Client_ID":"C008",
        "Need":"Enrobée",
        "Volume_t":13.61,
        "Volume_kg":13605.94,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8297649991,
        "Longitude":2.3169421334
    },
    {
        "Client_ID":"C009",
        "Need":"Enrobée",
        "Volume_t":14.16,
        "Volume_kg":14164.58,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8632249724,
        "Longitude":2.3568582611
    },
    {
        "Client_ID":"C010",
        "Need":"Waste",
        "Volume_t":12.89,
        "Volume_kg":12893.21,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8357622912,
        "Longitude":2.4148355918
    },
    {
        "Client_ID":"C011",
        "Need":"Waste",
        "Volume_t":13.35,
        "Volume_kg":13348.32,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.885563268,
        "Longitude":2.4053477927
    },
    {
        "Client_ID":"C012",
        "Need":"Waste",
        "Volume_t":10.18,
        "Volume_kg":10182.27,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8344111745,
        "Longitude":2.3409284004
    },
    {
        "Client_ID":"C013",
        "Need":"Waste",
        "Volume_t":18.69,
        "Volume_kg":18693.3,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8444315354,
        "Longitude":2.2819629252
    },
    {
        "Client_ID":"C014",
        "Need":"Waste",
        "Volume_t":14.21,
        "Volume_kg":14212.6,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8882465175,
        "Longitude":2.264484004
    },
    {
        "Client_ID":"C015",
        "Need":"Waste",
        "Volume_t":12.43,
        "Volume_kg":12430.33,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8567207885,
        "Longitude":2.2987460852
    },
    {
        "Client_ID":"C016",
        "Need":"Enrobée",
        "Volume_t":15.98,
        "Volume_kg":15981.13,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8360376647,
        "Longitude":2.3018284564
    },
    {
        "Client_ID":"C017",
        "Need":"Waste",
        "Volume_t":18.58,
        "Volume_kg":18577.18,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8563111256,
        "Longitude":2.4515215805
    },
    {
        "Client_ID":"C018",
        "Need":"Waste",
        "Volume_t":13.54,
        "Volume_kg":13536.02,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8843489829,
        "Longitude":2.3179995845
    },
    {
        "Client_ID":"C019",
        "Need":"Waste",
        "Volume_t":14.38,
        "Volume_kg":14380.32,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8277019945,
        "Longitude":2.3644008554
    },
    {
        "Client_ID":"C020",
        "Need":"Waste",
        "Volume_t":19.88,
        "Volume_kg":19877.06,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.852508881,
        "Longitude":2.4209095852
    },
    {
        "Client_ID":"C021",
        "Need":"Waste",
        "Volume_t":15.71,
        "Volume_kg":15708.49,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8958473007,
        "Longitude":2.2283797347
    },
    {
        "Client_ID":"C022",
        "Need":"Waste",
        "Volume_t":17.75,
        "Volume_kg":17747.05,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8840654658,
        "Longitude":2.3356365874
    },
    {
        "Client_ID":"C023",
        "Need":"Waste",
        "Volume_t":11.92,
        "Volume_kg":11916.03,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8998471212,
        "Longitude":2.322535139
    },
    {
        "Client_ID":"C024",
        "Need":"Waste",
        "Volume_t":12.86,
        "Volume_kg":12862.85,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8930421522,
        "Longitude":2.3358789851
    },
    {
        "Client_ID":"C025",
        "Need":"Enrobée",
        "Volume_t":17.04,
        "Volume_kg":17044.63,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8509098728,
        "Longitude":2.2290630239
    },
    {
        "Client_ID":"C026",
        "Need":"Waste",
        "Volume_t":13.99,
        "Volume_kg":13990.61,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8945021227,
        "Longitude":2.2973676485
    },
    {
        "Client_ID":"C027",
        "Need":"Waste",
        "Volume_t":14.33,
        "Volume_kg":14333.43,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8570057014,
        "Longitude":2.2757384372
    },
    {
        "Client_ID":"C028",
        "Need":"Waste",
        "Volume_t":10.44,
        "Volume_kg":10438.7,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8295147289,
        "Longitude":2.3976309301
    },
    {
        "Client_ID":"C029",
        "Need":"Enrobée",
        "Volume_t":15.05,
        "Volume_kg":15053.1,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8313453015,
        "Longitude":2.2891161603
    },
    {
        "Client_ID":"C030",
        "Need":"Waste",
        "Volume_t":17.0,
        "Volume_kg":17002.53,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8546398289,
        "Longitude":2.3367049267
    },
    {
        "Client_ID":"C031",
        "Need":"Waste",
        "Volume_t":19.75,
        "Volume_kg":19748.83,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8764464984,
        "Longitude":2.387878084
    },
    {
        "Client_ID":"C032",
        "Need":"Waste",
        "Volume_t":14.83,
        "Volume_kg":14832.12,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.888965554,
        "Longitude":2.2507665485
    },
    {
        "Client_ID":"C033",
        "Need":"Enrobée",
        "Volume_t":13.98,
        "Volume_kg":13976.26,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8634868191,
        "Longitude":2.463648015
    },
    {
        "Client_ID":"C034",
        "Need":"Waste",
        "Volume_t":15.17,
        "Volume_kg":15174.54,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8325553338,
        "Longitude":2.2648644966
    },
    {
        "Client_ID":"C035",
        "Need":"Waste",
        "Volume_t":11.15,
        "Volume_kg":11150.33,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8491311632,
        "Longitude":2.4134520825
    },
    {
        "Client_ID":"C036",
        "Need":"Enrobée",
        "Volume_t":16.84,
        "Volume_kg":16836.07,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8224189719,
        "Longitude":2.2367828582
    },
    {
        "Client_ID":"C037",
        "Need":"Waste",
        "Volume_t":14.0,
        "Volume_kg":13998.22,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.85201415,
        "Longitude":2.342531956
    },
    {
        "Client_ID":"C038",
        "Need":"Enrobée",
        "Volume_t":16.83,
        "Volume_kg":16825.14,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8511520307,
        "Longitude":2.4396337885
    },
    {
        "Client_ID":"C039",
        "Need":"Grue",
        "Volume_t":15.75,
        "Volume_kg":15745.92,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8288579155,
        "Longitude":2.3021950667
    },
    {
        "Client_ID":"C040",
        "Need":"Waste",
        "Volume_t":13.89,
        "Volume_kg":13892.92,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8372274366,
        "Longitude":2.4186360279
    },
    {
        "Client_ID":"C041",
        "Need":"Grue",
        "Volume_t":10.22,
        "Volume_kg":10215.16,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8986725908,
        "Longitude":2.2921971364
    },
    {
        "Client_ID":"C042",
        "Need":"Grue",
        "Volume_t":10.03,
        "Volume_kg":10031.65,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8598342054,
        "Longitude":2.3115851055
    },
    {
        "Client_ID":"C043",
        "Need":"Waste",
        "Volume_t":19.93,
        "Volume_kg":19929.27,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8210212275,
        "Longitude":2.2962086898
    },
    {
        "Client_ID":"C044",
        "Need":"Waste",
        "Volume_t":11.96,
        "Volume_kg":11960.49,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8762368303,
        "Longitude":2.2660315935
    },
    {
        "Client_ID":"C045",
        "Need":"Enrobée",
        "Volume_t":15.01,
        "Volume_kg":15014.12,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8872887819,
        "Longitude":2.4086223577
    },
    {
        "Client_ID":"C046",
        "Need":"Enrobée",
        "Volume_t":13.72,
        "Volume_kg":13724.74,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8674790585,
        "Longitude":2.313550389
    },
    {
        "Client_ID":"C047",
        "Need":"Waste",
        "Volume_t":19.88,
        "Volume_kg":19877.12,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8753450245,
        "Longitude":2.2514940748
    },
    {
        "Client_ID":"C048",
        "Need":"Waste",
        "Volume_t":17.63,
        "Volume_kg":17626.71,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8295176506,
        "Longitude":2.2938424521
    },
    {
        "Client_ID":"C049",
        "Need":"Enrobée",
        "Volume_t":17.72,
        "Volume_kg":17715.78,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8431161596,
        "Longitude":2.3191251583
    },
    {
        "Client_ID":"C050",
        "Need":"Waste",
        "Volume_t":18.67,
        "Volume_kg":18668.69,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8664786498,
        "Longitude":2.2519575393
    }
]

""".trimIndent()


In [30]:
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue

val mapper = jacksonObjectMapper()

val clients: List<Client> = mapper.readValue(clientsJson)

println("Loaded ${clients.size} clients:")
clients.forEach { println(it) }


Loaded 50 clients:
C001
C002
C003
C004
C005
C006
C007
C008
C009
C010
C011
C012
C013
C014
C015
C016
C017
C018
C019
C020
C021
C022
C023
C024
C025
C026
C027
C028
C029
C030
C031
C032
C033
C034
C035
C036
C037
C038
C039
C040
C041
C042
C043
C044
C045
C046
C047
C048
C049
C050


In [33]:
// Data classes

// Represents a geographical location
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    // Calculate Euclidean distance to another location
    fun distanceTo(other: Location): Double {
        val xDiff = latitude - other.latitude
        val yDiff = longitude - other.longitude
        return Math.sqrt(xDiff * xDiff + yDiff * yDiff)
    }
}

// Represents a client with specific needs
data class Client(
    val clientId: String,
    val need: String,
    val volumeTons: Double,
    val volumeKg: Double,
    val accessDifficulty: Boolean,
    val timeWindowLabel: String,
    val timeWindowStart: LocalTime,
    val timeWindowEnd: LocalTime,
    val location: Location
) {
    override fun toString() = clientId
}

// Represents the equipment that can be equipped on trucks
data class Equipment(
    val name: String,
    val capacityKg: Double
)

// Represents a truck
data class Truck(
    val name: String,
    val type: String,
    val maxCapacityKg: Double,
    val equipments: List<Equipment>,
    val canAccessDifficultAreas: Boolean,
    val homeLocation: Location
) {
    override fun toString() = name
}


In [34]:
// Import JSON data and map it to Client objects

import com.fasterxml.jackson.annotation.JsonFormat
import java.time.format.DateTimeFormatter

val clientsJson = """
[Your JSON data here]
""" // Replace with your actual JSON data

// Custom deserializer for LocalTime
val timeFormatter = DateTimeFormatter.ofPattern("HH:mm")

data class ClientJson(
    val Client_ID: String,
    val Need: String,
    val Volume_t: Double,
    val Volume_kg: Double,
    val Access_Difficulty: String,
    val Time_Window_Label: String,
    val Time_Window_Start: String,
    val Time_Window_End: String,
    val Latitude: Double,
    val Longitude: Double
)

val mapper = jacksonObjectMapper()
val clientsJsonList: List<ClientJson> = mapper.readValue(clientsJson)

// Map ClientJson to Client
val clients: List<Client> = clientsJsonList.map {
    Client(
        clientId = it.Client_ID,
        need = it.Need,
        volumeTons = it.Volume_t,
        volumeKg = it.Volume_kg,
        accessDifficulty = it.Access_Difficulty.equals("Yes", ignoreCase = true),
        timeWindowLabel = it.Time_Window_Label,
        timeWindowStart = LocalTime.parse(it.Time_Window_Start, timeFormatter),
        timeWindowEnd = LocalTime.parse(it.Time_Window_End, timeFormatter),
        location = Location(it.Latitude, it.Longitude)
    )
}

println("Loaded ${clients.size} clients:")
clients.forEach { println(it) }


Unrecognized token 'Your': was expecting (JSON String, Number, Array, Object or token 'null', 'true' or 'false')
 at [Source: REDACTED (`StreamReadFeature.INCLUDE_SOURCE_IN_LOCATION` disabled); line: 2, column: 6]
com.fasterxml.jackson.core.JsonParseException: Unrecognized token 'Your': was expecting (JSON String, Number, Array, Object or token 'null', 'true' or 'false')
 at [Source: REDACTED (`StreamReadFeature.INCLUDE_SOURCE_IN_LOCATION` disabled); line: 2, column: 6]
	at com.fasterxml.jackson.core.JsonParser._constructError(JsonParser.java:2567)
	at com.fasterxml.jackson.core.JsonParser._constructReadException(JsonParser.java:2593)
	at com.fasterxml.jackson.core.JsonParser._constructReadException(JsonParser.java:2601)
	at com.fasterxml.jackson.core.base.ParserMinimalBase._reportError(ParserMinimalBase.java:765)
	at com.fasterxml.jackson.core.json.ReaderBasedJsonParser._reportInvalidToken(ReaderBasedJsonParser.java:3018)
	at com.fasterxml.jackson.core.json.ReaderBasedJsonParser._hand